In [1]:
import face_recognition
import cv2
import os
import glob
import numpy as np
import time
import pickle
from Facerec import Facerec

In [4]:
validation_path="D:\\si\\Faces for Training\\Validation"

In [5]:
fr=Facerec()

In [25]:
images_path = glob.glob(os.path.join("D:\\si\\Faces for Training\\Data for Testing", "*.*"))
flag=0
for img_path in images_path:
    img=cv2.imread(img_path)
    box,name,acc=fr.detect_known_faces(img)
    if len(box)==0:
        save_path=os.path.join(validation_path,"UnableToDetect")
        if not(os.path.isdir(save_path)):
            os.mkdir(save_path)
        save=os.path.join(save_path,bname)
        cv2.imwrite(save,img)
    else:  
        for i in range(len(name)):
            if name[i]!="Unknown":
                box=box[i]
                name=name[i]
                
                acc=acc[i]
                flag=1
                break
        if flag==0:
            box=box[0]
            name=name[0]
            acc=acc[0]
            
        bname=os.path.basename(img_path)
        img=cv2.rectangle(img,(box[3],box[0]),(box[1],box[2]),(0,255,0),2)
        img=cv2.putText(img,"{0:.2f}".format(acc*100),(box[3],box[0]-10), cv2.FONT_HERSHEY_DUPLEX, 1,(0,255,0),2)
        save_path=os.path.join(validation_path,name)
        if not(os.path.isdir(save_path)):
            os.mkdir(save_path)
        save=os.path.join(save_path,bname)
        cv2.imwrite(save,img)
        flag=0
    
    